### 1. IMPORTS AND LIBRARIES

In [1]:
from keras.applications import VGG16
from keras.applications import ResNet50
from keras.applications import VGG19
from keras_applications.inception_resnet_v2 import InceptionResNetV2
import pandas as pd
import numpy as np
import tables

# Libraries for displying the data. 
from IPython.core.display import HTML 
from ipywidgets import interact
from IPython.display import display
from IPython.display import Image

print(VGG16.__doc__)
print(ResNet50.__doc__)
print(VGG19.__doc__)

Using TensorFlow backend.


None
None
None


In [2]:
# i am trying to use tensor-gpu just to check if things are fine
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess)

In [3]:
from skimage.io import imread
from skimage.transform import resize
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

### 2. HELPER FUNCTIONS

In [4]:
size = 224
def get_input(img_path):
    img = imread(img_path)
    img = resize(img, (size, size,3), preserve_range=True).astype(np.float32)
    #img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [5]:
def get_predictions(img_path, model,decode=True):
    input_ = get_input(img_path)
    out = model.predict(input_)
    if decode:
        return decode_predictions(out)
    else:
        return out

### 3. LOAD COMPLETE DATA 

In [6]:
df = pd.read_csv('../data/model/recommender_data.csv') 
df = df.drop(columns = 'Unnamed: 0')
print(df.shape)
df.head(2)

(35965, 13)


,image_link,product_name,page_link,label,background,delete,item_no,description,style,material,type_class,color,rating
0,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa,0,0.0,20370480,"Inspired by 1950's decor, the Brandbu futon is...","Mid-Century Modern, Scandinavian","Foam, Linen, Wood","Futon Set, Sofa","Blue, Green, Grey, Pink, Yellow",4.1
1,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa,0,0.0,20370480,"Inspired by 1950's decor, the Brandbu futon is...","Mid-Century Modern, Scandinavian","Foam, Linen, Wood","Futon Set, Sofa","Blue, Green, Grey, Pink, Yellow",4.1


In [7]:
df['label'].value_counts()

coffee table    12543
sofa             9680
tv stand         7033
pouf             3402
armchair         3307
Name: label, dtype: int64

### 5. LOAD MY MODEL AND GET EXTRACTABLE LAYER

In [8]:
from keras import models, layers, optimizers
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import keras
from keras import models, layers, optimizers
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [9]:
# load best model
my_model=models.load_model('weights.15-1.41.hdf5')

In [10]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
def make_preds(XR):
    feature=my_model_extract.predict(XR)
    df_output = pd.DataFrame(feature, columns = ['ifeature'+str(i) for i in range (1,neurons+1) ])
    return df_output 

In [12]:
# set the following
layer_to_extract = 'fc1'
neurons = 128

In [13]:
my_model_extract = Model(inputs=my_model.input, outputs=my_model.get_layer(layer_to_extract).output)

### 5. LOAD IMAGE MATRICES IN PARTS AND PREDICT

In [14]:
hdf5_file = tables.open_file('../data/reco_part1.h5', mode='r')
XR1 = np.array(hdf5_file.root.X_reco_part1)
hdf5_file.close()

In [15]:
hdf5_file = tables.open_file('../data/reco_part2.h5', mode='r')
XR2 = np.array(hdf5_file.root.X_reco_part2)
hdf5_file.close()

In [16]:
hdf5_file = tables.open_file('../data/reco_part3.h5', mode='r')
XR3 = np.array(hdf5_file.root.X_reco_part3)
hdf5_file.close()

In [17]:
hdf5_file = tables.open_file('../data/reco_part4.h5', mode='r')
XR4 = np.array(hdf5_file.root.X_reco_part4)
hdf5_file.close()

In [19]:
print(XR1.shape)
print(XR2.shape)
print(XR3.shape)
print(XR4.shape)

(8001, 224, 224, 3)
(9000, 224, 224, 3)
(10000, 224, 224, 3)
(8964, 224, 224, 3)


In [20]:
df_reco1=make_preds(XR1)
df_reco1.shape

(8001, 128)

In [21]:
# df_reco1.to_csv('../data/temp_R1.csv')

In [22]:
df_reco2=make_preds(XR2)
df_reco2.shape

(9000, 128)

In [23]:
# df_reco2.to_csv('../data/temp_R2.csv')

In [24]:
df_reco3=make_preds(XR3)
df_reco3.shape

(10000, 128)

In [25]:
# df_reco3.to_csv('../data/temp_R3.csv')

In [26]:
df_reco4=make_preds(XR4)
df_reco4.shape

(8964, 128)

In [27]:
# df_reco4.to_csv('../data/temp_R4.csv')

### 6. CONCAT AND MAKE FINAL RECOMMENDATION MATRIX

In [28]:
df['label'].value_counts()

coffee table    12543
sofa             9680
tv stand         7033
pouf             3402
armchair         3307
Name: label, dtype: int64

In [30]:
df.iloc[:,:4].head()

,image_link,product_name,page_link,label
0,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa
1,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa
2,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa
3,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa
4,https://ak1.ostkcdn.com/images/products/139907...,Carson Carrington Brandbu Mid-century Fold Dow...,https://www.overstock.com/Home-Garden/Carson-C...,sofa


In [31]:
df_reco_1 = pd.concat([df_reco1,df_reco2,df_reco3,df_reco4]).reset_index()

In [32]:
df_reco_1.shape

(35965, 129)

In [33]:
df_reco_1.insert(loc=0, column='label', value=df['label'])
df_reco_1.insert(loc=0, column='page_link', value=df['page_link'])
df_reco_1.insert(loc=0, column='product_name', value=df['product_name'])
df_reco_1.insert(loc=0, column='image_link', value=df['image_link'])

In [34]:
df_reco1.head()

,ifeature1,ifeature2,ifeature3,ifeature4,ifeature5,ifeature6,ifeature7,ifeature8,ifeature9,ifeature10,...,ifeature119,ifeature120,ifeature121,ifeature122,ifeature123,ifeature124,ifeature125,ifeature126,ifeature127,ifeature128
0,0.000000,0.0,0.0,0.0,809.768860,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,645.975525
1,0.000000,0.0,0.0,0.0,606.452271,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,513.054871
2,0.000000,0.0,0.0,0.0,632.849670,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.953590,0.0,0.0,0.0,0.0,436.987091
3,23.811977,0.0,0.0,0.0,528.771545,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,43.345352,0.0,0.0,0.0,0.0,389.577667
4,48.486305,0.0,0.0,0.0,531.305237,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.215407,0.0,0.0,0.0,0.0,353.925629


In [35]:
df_reco_1['label'].value_counts()

coffee table    12543
sofa             9680
tv stand         7033
pouf             3402
armchair         3307
Name: label, dtype: int64

In [36]:
df_reco_1.to_csv('../data/recommender_matrix.csv')

In [ ]:
##End here

In [ ]:
#pull_out=df_recommender.loc[0,:]
pic_link = '../data/samples/light-grey-sofa.jpeg'
print(pic_link)
display(Image(filename=pic_link,width=300, height=300))

In [ ]:
# matrices the input image
user_img_matrix=get_input(pic_link)


In [ ]:
user_img_matrix = np.expand_dims(user_img_matrix, axis=0)
user_img_matrix.shape

In [ ]:
user_data = model_extract.predict(user_img_matrix)

In [ ]:
search_in = np.array(df_recommender.iloc[:,4:])

In [ ]:
print(user_data.shape)
print(search_in.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
results = cosine_similarity(search_in, user_data)

In [ ]:
show_me = pd.DataFrame(results).sort_values(0, ascending=False).head(5)
show_me

In [ ]:
recommend = df_recommender.iloc[list(show_me.index),0:4].values
print(recommend)

In [ ]:
for i in range(0,5):    
    display(Image(url=recommend[i][0],width=244, height=244))
    print('Link to page =',  recommend[i][2])